In [1]:
import numpy as np
import random
import tensorflow as tf

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarning: Passing (type, 1) or 

In [52]:
#Define Board
class TicTacToeBoard:
    def __init__(self, boardHeightWidth):

        #Hyper-parameters to be used with the neural network agent
        #Defines the rewards to be used i nthe Q function
        self.winReward = 10
        self.loseReward = -1
        self.drawReward = 1;
        self.moveReward = 0;
        self.illegalMoveReward = -1
        
        #The tic tac toe board will always be square the have a 1D lenght of this
        self.boardHeightWidth = boardHeightWidth
        
        #Define a blank game state board. The board must always be square. The stateBoard however
        #will have 2 boards contained it is as defined by an M x 2N array.In a regular 3 row/col
        #tic tac toe board this would mean there will be a 3 x 6 array. The first 3 rows containig
        #entries for the X player and the last 3 rows containing entries for the O player.
        #The initial state of the board will be all zeros
        self.stateBoard = np.zeros([boardHeightWidth * 2, boardHeightWidth], dtype=int)
        
    def CheckWinCondition(self):
        #Given the current state of the game board, checks  to see if a winning move has been made    
        #Initialize winner. -1: No winner, 0: X wins, 1: O wins

        #Check if 3 in a row horizontally    
        for row in range(boardHeightWidth):
            #Reset counters
            xCounter = 0;
            oCounter = 0;
            for col in range(boardHeightWidth):
                if self.stateBoard[row][col] == 1:
                    xCounter += 1
                if self.stateBoard[row + self.boardHeightWidth][col] == 1:
                    oCounter += 1
            if xCounter == self.boardHeightWidth:
                winner = 0
                #display("Winner by Horizonal")
                return winner
            elif oCounter == self.boardHeightWidth:
                winner = 1
                #display("Winner by Horizontal")
                return winner

        #Check if 3 in a row vertically
        for col in range(self.boardHeightWidth):
            #Reset counters
            xCounter = 0;
            oCounter = 0;
            for row in range(self.boardHeightWidth):
                if self.stateBoard[row][col] == 1:
                    xCounter += 1
                if self.stateBoard[row + self.boardHeightWidth][col] == 1:
                    oCounter += 1
            if xCounter == self.boardHeightWidth:
                winner = 0
                return winner
            elif oCounter == self.boardHeightWidth:
                winner = 1
                return winner
 

        #Check left to right diagonal
        for colrow in range(self.boardHeightWidth):
            #Reset counter 
            xCounter = 0
            oCounter = 0
            
            if self.stateBoard[colrow][colrow] == 1:
                 xCounter += 1
            if self.stateBoard[colrow + self.boardHeightWidth][colrow] == 1:
                 oCounter += 1                         
        if xCounter == self.boardHeightWidth:
            winner = 1
            return winner
        if oCounter == self.boardHeightWidth:
            winner = 0
            return winner
        
        
        #Reset counter 
        xCounter = 0
        oCounter = 0
        
        #Check right to left diagonal
        colList = np.arange(self.boardHeightWidth-1, -1, -1).tolist()
        for row in range(self.boardHeightWidth):
            col = colList[row]
            if self.stateBoard[row][col] == 1:
                xCounter += 1
            if self.stateBoard[row + self.boardHeightWidth][col] == 1:
                oCounter += 1
        if xCounter == self.boardHeightWidth:
            winner = 1
            return winner
        if oCounter == self.boardHeightWidth:
            winner = 0
            return winner
        
        #If we make it here, there is no winner
        return -1
    
    def CheckDrawCondition(self):
        #Check to see if the board is completely full
        
        #Grab the compressed board which maries the X and O's state board
        compressedBoard = self.CompressedBoard()
        
        #If the number of ones contained in the compressed board is equal to the number of squares
        #on the game board then its a draw
        if np.count_nonzero(compressedBoard) == (self.boardHeightWidth * self.boardHeightWidth):      
            return True
        else:
            return False
    
    def MakeMove(self, row, col, playerNum):
        #Each agent playing the game will call this function to make their move
        #during their turn by entering the row and col they want to mark on th game board
        #playerNum 0 = X, 1 = O
        
        #Check if the player is trying to place a mark in a column that is already
        #occupied by a mark
        if self.CheckIllegalMove(row, col) == True:
            display("Move by player: " + str(playerNum) + " was illegal")
            return self.illegalMoveReward
        
        #Place their mark
        if playerNum == 0:
            self.stateBoard[row][col] = 1
        elif playerNum == 1:
            row = row + self.boardHeightWidth
            self.stateBoard[row][col] = 1
            
        #Check if the move resulted in a game ending condition    
        #Return the proper award if so
        winner = self.CheckWinCondition()
        if winner == 0 and playerNum == 0:
            return self.winReward
        elif winner == 1 and playerNum == 1:
            return self.winReward
        elif self.CheckDrawCondition() == True:
            return self.drawReward
        else:
            return self.moveReward
    
    def CheckIllegalMove(self, row, col):
        #sChecks if the player is trying to place a mark in a column that is already
        #occupied by a mark
        #Returns a boolean value indicating if the move is illegal
        
        #Split stateBoard into player components
        splitBoard = np.split(self.stateBoard,2, axis=0)
        xBoard = splitBoard[0]
        oBoard = splitBoard[1]
        combinedBoard = xBoard|oBoard
        
        if combinedBoard[row][col] == 1:
            return True
        else:
            return False
        
    def CompressedBoard(self):
    #Returns a stateBoard which has been collapsed down to a game board to show which spaces have something
    #in them and which are empty. Takes a M x 2N array, splits it in half along the N axis and bitwise ors
    #the two into an MxN array
        splitBoard = np.split(self.stateBoard,2, axis=0)
        xBoard = splitBoard[0]
        oBoard = splitBoard[1]
        combinedBoard = xBoard|oBoard
        return combinedBoard
    
    def ResetBoard(self):
        #Resets the state board at the end of each game
        self.stateBoard = np.zeros([boardHeightWidth * 2, boardHeightWidth], dtype=int)

In [4]:
class RandomPlayer:
#Define the random player
#This player will only make random moves on the board

    def MakeMove(self, combinedStateBoard):  
    #Decides which squar it should fill at random given what is not
    #already occupied. Return the row, col to be played
       
        #Find empty indices
        indices = np.argwhere(combinedBoard == 0)
        move = random.choice(indices)
        return move[0], move[1]
 

In [20]:
# QNeural Network Q Learning Player

class NNPlayer:
    def __init__(self, learningRate , discount, explorationRate, boardHeightWidth, numGames):    
        
        #Q Learning Hyper Parameters
        self.discount = discount
        self.learningRate = learningRate
        self.discount = discount
        self.explorationRate = explorationRate
        self.explorationDelta = 1.0 / numGames    
        
        #Keep track of board dimensions for internal calculations
        self.boardHeightWidth = boardHeightWidth
        
        #Neural Network Parameters
        #Input layer will be the size of the  state board(game board x 2) since we track X e
        #ntries and O entries indifferent parts of the array
        self.inputLayerSize = 2 * boardHeightWidth * boardHeightWidth
        #Output layer will be the size of the gameboard and indicate which square to fill
        self.outputLayerSize = boardHeightWidth * boardHeightWidth
        
        #Set up tensorflow netwrok
        self.session = tf.Session()
        self.AssembleNetwork()
        self.session.run(self.initializer)
        
        
    def AssembleNetwork(self):
        #Create the neural network architecture
        
        #Define how many neurons should be in each hidden layer
        #For this exercise i chose to start testing with 2 layers 
        #starting off bigger in size than the input layer and
        #tapering down a little in the second layer
        layer1Size = self.inputLayerSize + 8
        layer2Size = self.inputLayerSize + 2
        
        #Variable to hold the input values
        self.nnInput = tf.placeholder(dtype = tf.float32, shape = [None, self.inputLayerSize])
        
        #Set up the hidden layers
        fc1 = tf.layers.dense(self.nnInput, layer1Size, activation = tf.sigmoid, kernel_initializer = tf.constant_initializer(np.zeros((self.inputLayerSize, layer1Size))))
        fc2 = tf.layers.dense(fc1, layer2Size, activation = tf.sigmoid, kernel_initializer = tf.constant_initializer(np.zeros((layer2Size, self.outputLayerSize))))
        
        self.nnOutput = tf.layers.dense(fc2, self.outputLayerSize)                     
        self.targetOutput = tf.placeholder(shape=[None, self.outputLayerSize], dtype=tf.float32)
        
        #Setup up the loss function to figure out how far we are off in accurary
        loss = tf.losses.mean_squared_error(self.targetOutput, self.nnOutput)
        
        #Setup our descent gradient optimizer
        self.optimizer = tf.train.GradientDescentOptimizer(learning_rate=self.learningRate).minimize(loss)
                              
        self.initializer = tf.global_variables_initializer()
        
    def getQ(self, stateBoard):
    
        #One hot encode out state board so it can be put into the network
        oneHotEncodedBoard = self.OneHotEncode(stateBoard)
        
        #Run the state board through the network and return the 1D array of output values
        return self.session.run(self.nnOutput, feed_dict={self.nnInput: oneHotEncodedBoard})[0]
    
    def OneHotEncode(self, stateBoard):
        #The state board is already in one hot format, just need to reshape it into a 1D array
        reshapedBoard = stateBoard.reshape(-1)
        reshapedBoard = reshapedBoard[None, :]
        return reshapedBoard
    
    def RandomMove(self, stateBoard):
        #Returns a row, col of a random move to be made given the available squared
        splitBoard = np.split(stateBoard,2, axis=0)
        xBoard = splitBoard[0]
        oBoard = splitBoard[1]

        combinedBoard = xBoard|oBoard
        indices = np.argwhere(combinedBoard == 0)
        move = random.choice(indices)
        return move[0], move[1]
    
    def GetMove(self, stateBoard):
    #Chooses which move to make
        
        #Exploration allows the neural net to avoid getting stuck in a rut based on intial weightings
        #If rand returns a number lower than the current exploration rate, perform a random move which 
        #will help the network explore the rewards of paths it has not seen yet
        if random.random() > self.explorationRate:
            #Calculate a move based on Q learning
            
            #Grab Q array for current stateBoard
            qArray = self.getQ(stateBoard)
            
            #Determine the set of legal moves
            splitBoard = np.split(stateBoard,2, axis=0)
            xBoard = splitBoard[0]
            oBoard = splitBoard[1]
            combinedBoard = xBoard|oBoard
            indices = np.argwhere(combinedBoard == 0)
            
            #Create a list of row, col pairs that are legal moves that can be made
            flattenedIndices =  list()
            for rcSet in indices:
                flattenedIndices.append(rcSet[0] * self.boardHeightWidth + rcSet[1])
            
            #Pick the index fromt the Q table which has the highest
            #value  for the given set of valid choices. 
            #flattenedIndex will be the 1D index of the move to be made 
            flattenedIndex = flattenedIndices[np.argmax(qArray[flattenedIndices])]
 
            
            #Unflatted this index into  a row, col set
            if flattenedIndex < self.boardHeightWidth - 1:
                col = flattenedIndex
                row = 0
            else:
                row =  flattenedIndex//self.boardHeightWidth
                col = (flattenedIndex % self.boardHeightWidth)
            
            return int(row), int(col)
        
        else:
            #Return a random move
            return self.RandomMove(stateBoard)
        
    def Train(self, oldStateBoard, newStateBoard, action, reward):
    #Trains the neural network according to the Q learning methedology
    #Takes in the state of the board before making a move as well as what move was
    #made, the state of the board after that move and the reward that was given for the move
        
        #Grab Q values for each state
        oldStateQValues = self.getQ(oldStateBoard)
        newStateQValues = self.getQ(newStateBoard)
        
        #Calculate the Q value given the future state, reward and discount hyper parameter
        oldStateQValues[action] = reward + self.discount * np.amax(newStateQValues)
        
        #Set up neural net trainig data 
        trainingInput = self.OneHotEncode(oldStateBoard)
        targetOutput = [oldStateQValues]
        trainingData = {self.nnInput: trainingInput, self.targetOutput: targetOutput}
        
        #Train the model given the current moves
        self.session.run(self.optimizer, feed_dict = trainingData)

    def Update(self, oldStateBoard, newStateBoard, row, col, reward):
        #Translate 2d row col values into a 1D index for th corresponding output layer index
        flattenedIndex = row * self.boardHeightWidth + col
        
        #Train the mode
        self.Train(oldStateBoard, newStateBoard, flattenedIndex, reward)
        
        #Lower the exploration rate. Early on, the model should do more random
        #exploration to stop stagnation but as the model begins to get its feet under it
        #lower that and let the q learnin take over
        if (self.explorationRate - self.explorationDelta) >= 0:
                self.explorationRate -= self.explorationDelta

In [60]:
#Main Program

#Q Learning Hyper-parameters
learningRate = .1
discount = .9
explorationRate = 1.0

#Number of games to train on
numGames = 1001

#Report out stastics of games won/lost/draw after this many games
reportingPeriod = 300

#Board dimensions
boardHeightWidth = 5

#Setup game board
GameBoard = TicTacToeBoard(boardHeightWidth)
gameOver = False

#Set up our players
RandomAgent = RandomPlayer()
NNAgent = NNPlayer(learningRate , discount, explorationRate, boardHeightWidth, numGames)

#Counters for keeping track of game statistics
overallWinCounter = np.array([0, 0])
subsectionWinCounter = np.array([0, 0])
overallDrawCounter = 0;
subsectionDrawCounter = 0;

for gameNum in range (numGames):
    #Reset Game Board
    GameBoard.ResetBoard()

    while True:
        
        #X makes move
        board = GameBoard.stateBoard
        oldStateBoard =  GameBoard.stateBoard
        combinedBoard = GameBoard.CompressedBoard()
        row, col = RandomAgent.MakeMove(combinedBoard)
        reward = GameBoard.MakeMove(row, col, 1)
        newStateBoard = GameBoard.stateBoard

        winner = GameBoard.CheckWinCondition()
        draw = GameBoard.CheckDrawCondition()
        if winner != -1:
            #print("We have a winner: Player " + str(winner))
            #display("Game # " + str(gameNum))
            #display(board)
            subsectionWinCounter[winner] += 1
            #If the opponent wins, the model needs to train itself on the loss
            #Gives the negative of the reward the opponent got
            NNAgent.Update(oldStateBoard, newStateBoard, nnRow, nnCol, -reward)
            break
        elif draw == True:
            #Game ended in draw
            #print("Game ended in draw")
            subsectionDrawCounter += 1
            break   
        
        #O makes move
        oldStateBoard =  GameBoard.stateBoard
        nnRow, nnCol = NNAgent.GetMove(GameBoard.stateBoard)
        reward = GameBoard.MakeMove(nnRow, nnCol, 0)
        newStateBoard = GameBoard.stateBoard
        NNAgent.Update(oldStateBoard, newStateBoard, nnRow, nnCol, reward)
        
        winner = GameBoard.CheckWinCondition()
        draw = GameBoard.CheckDrawCondition()
        if winner != -1:
            #print("We have a winner: Player " + str(winner))
            #display("Game # " + str(gameNum))
            #isplay(board)
            subsectionWinCounter[winner] += 1
            break
        elif draw == True:
            #print("Game ended in draw")
            subsectionDrawCounter += 1
            break   
        
    #Report out game statistics per the reporting period
    if gameNum % reportingPeriod == 0 and gameNum != 0:
        overallWinCounter[0] += subsectionWinCounter[0]
        overallWinCounter[1] += subsectionWinCounter[1]
        overallDrawCounter += subsectionDrawCounter
        display("Stats for the past " + str(reportingPeriod) + " games")
        display("X's won: " + str(subsectionWinCounter[0]) + "  O's won: " + str(subsectionWinCounter[1]) + "  Draws:" + str(subsectionDrawCounter)) 
        display("Percentage X's won: " + str((subsectionWinCounter[0]/reportingPeriod)*100) + "%  Percentage O's won: " + str((subsectionWinCounter[1]/reportingPeriod)*100) + "%  Percentage Draws:" + str((subsectionDrawCounter/reportingPeriod)*100) + "%") 
        
        #Reset the reporting period specific counters
        subsectionWinCounter[0] = 0
        subsectionWinCounter[1] = 0
        subsectionDrawCounter = 0
        
#Report out overall stats across all games
display("Stats for the overal run of games")
display("X's won: " + str(overallWinCounter[0]) + "  O's won: " + str(overallWinCounter[1]) + "  Draws:" + str(overallDrawCounter))

'Stats for the past 300 games'

"X's won: 100  O's won: 51  Draws:150"

"Percentage X's won: 33.33333333333333%  Percentage O's won: 17.0%  Percentage Draws:50.0%"

'Stats for the past 300 games'

"X's won: 118  O's won: 54  Draws:128"

...

'Stats for the past 300 games'

"X's won: 128  O's won: 57  Draws:115"

"Percentage X's won: 42.66666666666667%  Percentage O's won: 19.0%  Percentage Draws:38.333333333333336%"

'Stats for the overal run of games'

"X's won: 346  O's won: 162  Draws:393"